In [ ]:
# loading data set

from datasets import load_dataset
dataset = load_dataset("d4rk3r/resumes-raw-pdf")
dataset.features
pdf_item = dataset[0]
print(f"PDF type: {type(pdf_item['pdf'])}")

In [ ]:
# loading web dataset from url

import webdataset as wds
url = "https://huggingface.co/datasets/pixparse/pdfa-eng-wds/resolve/main/data/pdfa-eng-train-0001.tar"
web_dataset = wds.WebDataset(url)

In [ ]:
# PDF data set byte format
# taking sample from data set
import json
from datasets import load_dataset

dataset = load_dataset("pixparse/pdfa-eng-wds", streaming=True)
small_dataset = dataset["train"].take(5)


# for doc in dataset["train"]:
#     print(len(doc["pdf"]))

Resolving data files:   0%|          | 0/1801 [00:00<?, ?it/s]

In [8]:
from src import *
for doc in small_dataset:
    pdf_doc = pdf_bytes_to_doc_stream(doc['pdf'], doc['__key__'])
    extract_elements_from_pdf(pdf_doc)
    

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 1839ac7b8f395811156cc7d3e509f667
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered picture descriptions: ['vlm', 'api']
INFO:docling.pipeline.base_pipeline:Processing document 6368667
c:\Users\adler\Desktop\rag_proj\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is

In [6]:
for doc in small_dataset:
    print(doc["__key__"])

6368667
4102114
7912120
7292019
3180753


In [ ]:
# small_dataset.features


items = list(small_dataset)
last_item = items[-1]
for page in last_item['json']['pages']:
    print(type(page))
    for item in page:
        print(item)
        print(type(item))

# pdf_bytes = last_item["pdf"]

In [ ]:
import requests
import json
def embed_on_ollama(text, model="bge-m3", ollama_url="http://25.12.119.117:11435", timeout=30):
    print(text)
    try:
        # Prepare the request payload
        payload = {
            "model": model,
            "prompt": str(text)
        }
        
        print(f"Sending request to {ollama_url}/api/embeddings...")
        print(payload)
        # Make the API request with timeout
        response = requests.post(
            f"{ollama_url}/api/embeddings",
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=timeout
        )
        
        
        print(f"Response status: {response.status_code}")
        
        # Check if the request was successful
        #response.raise_for_status()
        
        print(response)
        
        # Parse the response and extract the embedding
        result = response.json()
        print("results:")
        print(result)
        embedding = result.get("embedding")
        
        if len(embedding) == 0:
            print("Error: No embedding found in response")
            print(f"Response content: {result}")
            return None
            
        print("Received embedding successfully")
        print(embedding)
        print(type(embedding))
        return embedding
        
    except requests.exceptions.Timeout:
        print(f"Request timed out after {timeout} seconds")
        return None
    except requests.exceptions.ConnectionError:
        print(f"Connection error - could not connect to {ollama_url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        print(f"Response text: {response.text}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None


In [37]:
embed_on_ollama("yee",model="bge-m3", ollama_url="http://25.12.119.117:11435", timeout=30)

yee
Sending request to http://25.12.119.117:11435/api/embeddings...
{'model': 'bge-m3', 'prompt': 'yee'}
Request timed out after 30 seconds


In [ ]:
# quick test if pdf is scanned

def quick_scan_check(pdf_bytes):

    import fitz
    
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    
    # בדיקת עמוד ראשון בלבד (למהירות)
    if len(doc) > 0:
        page = doc[0]
        text = page.get_text().strip()
        images = page.get_images()
        
        doc.close()
        
        # אם אין טקסט אבל יש תמונות - כנראה סרוק
        if len(text) < 10 and len(images) > 0:
            return True, "Likely scanned - minimal text, has images"
        elif len(text) > 100:
            return False, "Has substantial text - not scanned"
        else:
            return None, "Unclear - needs deeper analysis"
    
    doc.close()
    return None, "Empty PDF"

is_scanned, reason = quick_scan_check(pdf_bytes)
print(f"Quick check - Is scanned: {is_scanned} ({reason})")

In [ ]:
document = result.document
print(result.document.name)
for x in document:
    print(x[0])
# print(f'{list(result.input)}')
# print(f'{list(result.model_fields_set)}')
# print(f'{list(result.pages)}')
# print(f'{list(result.status)}')
# print(f'{list(result.timings)}')

In [ ]:
def print_document_content(document):
    metadata = {"name": document.name,
                "version": document.version,
                "origin": document.origin,
                "number_of_pages": len(document.pages),
                "text_count": len(document.texts),
                "picture_count": len(document.pictures),
                "table_count": len(document.tables)
                }
    
    if document.texts:
        text_matadata = []
        for text in document.texts:
            metadata = {"id": text.self_ref,
                        "parent": text.parent.cref,
                        "children": [c.cref for c in text.children],
                        "orig": text.orig,
                        "text": text.text,
                        "formatting": text.formatting,
                        "hyperlink": text.hyperlink}
            text_matadata.append(metadata)

    if document.pictures:
        pic_metadata = []
        for pic in document.pictures:
            metadata = {"id": pic.self_ref,
                        "parent": pic.parent.cref,
                        "children": [c.cref for c in pic.children],
                        "captions":[c.cref for c in pic.captions],
                        "references":pic.refereces,
                        "footnotes": pic.footnotes,
                        "image": pic.image,
                        "annotations": pic.annotations}
            pic_metadata.append(metadata)

    

    if document.tables:
        table_metadata = []
        for table in document.tables:
            metadata = {
                            "id": table.self_ref,
                            "parent": table.parent.cref,
                            "children": [c.cref for c in table.children],
                            "captions":[c.cref for c in table.captions],
                            "references":table.refereces,
                            "footnotes": table.footnotes,
                            "image": table.image,
                            "annotations": table.annotations}
            table_metadata.append(metadata)
            
    metadata = {"name": document.name,
            "version": document.version,
            "origin": document.origin,
            "number_of_pages": len(document.pages),
            "text_count": len(document.texts),
            "picture_count": len(document.pictures),
            "table_count": len(document.tables),
            "text_metadata": text_matadata,
            "picture_metadata": pic_metadata,
            "tables_metadata": table_metadata
            }
    
    # # זוגות מפתח-ערך
    # if document.key_value_items:
    #     print(f"🔑 זוגות מפתח-ערך ({len(document.key_value_items)} פריטים):")
    #     print("-" * 30)
    #     for i, kv in enumerate(document.key_value_items, 1):
    #         print(f"  {i}. {kv}")
    #     print()

#     # אלמנטי טפסים
#     if document.form_items:
#         print(f"📋 אלמנטי טפסים ({len(document.form_items)} פריטים):")
#         print("-" * 30)
#         for i, form in enumerate(document.form_items, 1):
#             print(f"  {i}. {form}")
#         print()
    
#     # קבוצות
#     if document.groups:
#         print(f"👥 קבוצות ({len(list(document.groups))} פריטים):")
#         print("-" * 30)
#         for i, group in enumerate(document.groups, 1):
#             print(f"  {i}. {group}")
#         print()
    
    
    # # עמודים
    # if document.pages:
    #     print(f"📄 עמודים ({len(document.pages)} פריטים):")
    #     print("-" * 30)
    #     for i, page in enumerate(document.pages):
    #         print(f"  {i}. {page}")
    #     print()
#     print("=" * 50)


# # דוגמה לשימוש:
print_document_content(document)

In [ ]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 2.0


logging.basicConfig(level=logging.INFO)

data_folder = Path("__file__").parent / "data"
input_doc_path = "https://arxiv.org/pdf/2408.09869"
output_dir = Path("scratch")

# Important: For operating with page images, we must keep them, otherwise the DocumentConverter
# will destroy them for cleaning up memory.
# This is done by setting PdfPipelineOptions.images_scale, which also defines the scale of images.
# scale=1 correspond of a standard 72 DPI image
# The PdfPipelineOptions.generate_* are the selectors for the document elements which will be enriched
# with the image field
pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True
# pipeline_options.do_ocr = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

start_time = time.time()

conv_res = doc_converter.convert(input_doc_path)

output_dir.mkdir(parents=True, exist_ok=True)
doc_filename = conv_res.input.file.stem

# Save page images
for page_no, page in conv_res.document.pages.items():
    page_no = page.page_no
    page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
    with page_image_filename.open("wb") as fp:
        page.image.pil_image.save(fp, format="PNG")

# Save images of figures and tables
table_counter = 0
picture_counter = 0
for element, _level in conv_res.document.iterate_items():
    if isinstance(element, TableItem):
        table_counter += 1
        element_image_filename = (
            output_dir / f"{doc_filename}-table-{table_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(conv_res.document).save(fp, "PNG")

    if isinstance(element, PictureItem):
        picture_counter += 1
        element_image_filename = (
            output_dir / f"{doc_filename}-picture-{picture_counter}.png"
        )
        with element_image_filename.open("wb") as fp:
            element.get_image(conv_res.document).save(fp, "PNG")

# Save markdown with embedded pictures
md_filename = output_dir / f"{doc_filename}-with-images.md"
conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

# Save markdown with externally referenced pictures
md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

# Save HTML with externally referenced pictures
html_filename = output_dir / f"{doc_filename}-with-image-refs.html"
conv_res.document.save_as_html(html_filename, image_mode=ImageRefMode.REFERENCED)

end_time = time.time() - start_time



In [ ]:
document = conv_res.document
print("=== כל הטקסטים ===")
for i, text_item in enumerate(document.texts):
    print(f"טקסט {i}: {text_item.text}")
    print(text_item.)
    print(f"סוג: {text_item.label}")
    if text_item.prov:  # מידע על מיקום
        bbox = text_item.prov[0].bbox
        print(f"מיקום: עמוד {text_item.prov[0].page_no}, (x: {bbox.l}, y: {bbox.t})")
    print("---")
    break

In [ ]:
def analyze_by_element_type(document):
    """ניתוח לפי סוג אלמנט"""
    from collections import Counter
    
    # ספירת סוגי אלמנטים
    element_types = Counter([text.label for text in document.texts])
    print("=== סוגי אלמנטים ===")
    for element_type, count in element_types.most_common():
        print(f"{element_type}: {count}")
    
    # בדיקה מפורטת לפי סוג
    print("\n=== דוגמאות לפי סוג ===")
    seen_types = set()
    for text in document.texts:
        if text.label not in seen_types:
            seen_types.add(text.label)
            print(f"\n{text.label}:")
            print(f"  דוגמה: {text.text[:100]}...")
            print(f"  תכונות: {[attr for attr in dir(text) if not attr.startswith('_') and not callable(getattr(text, attr))]}")
            
            # בדיקה מיוחדת לסוגים שיכולים להכיל הפניות
            if text.label in ['reference', 'footnote', 'caption']:
                print(f"  ⭐ מצאתי {text}!")

analyze_by_element_type(document)

In [ ]:
print(f"טקסטים: {len(document.texts)}")
print(f"טבלאות: {len(document.tables)}")
print(f"תמונות: {len(document.pictures)}")
print(f"קבוצות: {len(document.groups)}")

In [ ]:
print(f"=== סה\"כ תמונות: {len(document.pictures)} ===")

for i, picture in enumerate(document.pictures):
    print(f"\n--- תמונה {i+1} ---")
    print(f"מזהה: {picture.self_ref}")
    print(f"סוג: {picture.label}")
    
    # כיתובים
    if hasattr(picture, 'captions') and picture.captions:
        print(f"כיתובים ({len(picture.captions)}):")
        for j, caption in enumerate(picture.captions):
            print(f"  {j+1}. {caption}")
    else:
        print("אין כיתובים")
    
    # הפניות
    if hasattr(picture, 'references') and picture.references:
        print(f"הפניות ({len(picture.references)}):")
        for j, ref in enumerate(picture.references):
            print(f"  {j+1}. {ref.text}")
    else:
        print("אין הפניות")
    
    # מיקום
    if hasattr(picture, 'prov') and picture.prov:
        prov = picture.prov[0]
        print(f"מיקום: עמוד {prov.page_no}")
    
    # כל התכונות הזמינות
    attrs = [attr for attr in dir(picture) if not attr.startswith('_')]
    print(f"תכונות זמינות: {attrs}")


In [ ]:
print("=== כל הטבלאות ===")
for i, table in enumerate(document.tables):
    print(f"טבלה {i+1}:")
    # print(f"תוכן: {table}")
    
    
    # גישה למבנה הטבלה (אם זמין)
    if hasattr(table, 'data') and table.data:
        print(f"מספר שורות: {len(table.data.table_cells)}")
        
    # מיקום הטבלה
    if table.prov:
        bbox = table.prov[0].bbox
        print(f"מיקום: עמוד {table.prov[0].page_no}")
    print("---")

In [ ]:
print("=== כל התמונות ===")
for i, picture in enumerate(document.pictures):
    print(f"תמונה {i+1}:")
    print(f"תווית: {picture.label}")
    
    # כיתובים
    if picture.captions:
        for caption in picture.captions:
            print(f"כיתוב: {caption}") #???
    
    # מיקום
    if picture.prov:
        bbox = picture.prov[0].bbox
        print(f"מיקום: עמוד {picture.prov[0].page_no}")
    print("---")

In [ ]:
# חיפוש כותרות
headers = [text for text in document.texts if text.label in ['section_header', 'page_header']]
print(f"מצאתי {len(headers)} כותרות")

# חיפוש רשימות
lists = [text for text in document.texts if text.label == 'list_item']
print(f"מצאתי {len(lists)} פריטי רשימה")

# חיפוש נוסחאות
formulas = [text for text in document.texts if text.label == 'formula']
print(f"מצאתי {len(formulas)} נוסחאות")

In [ ]:


# בחינת מטא דאטה של טקסט
for i, text_item in enumerate(document.texts[:3]):  # 3 הראשונים
    print(f"=== טקסט {i+1} ===")
    print(f"תוכן: {text_item.text[:100]}...")
    print(f"סוג: {text_item.label}")
    print(f"הורה: {text_item.parent}")
    print(f"ילדים: {len(text_item.children)}")
    
    # מידע מיקום
    if text_item.prov:
        prov = text_item.prov[0]
        print(f"עמוד: {prov.page_no}")
        print(f"מיקום: x={prov.bbox.l:.1f}, y={prov.bbox.t:.1f}")
        print(f"גודל: רוחב={prov.bbox.r-prov.bbox.l:.1f}, גובה={prov.bbox.b-prov.bbox.t:.1f}")
        print(f"טווח תווים: {prov.charspan}")
    
    # מידע נוסף
    if text_item.:
        print(f"כיתובים: {len(text_item.captions)}")
    if text_item.references:
        print(f"הפניות: {len(text_item.references)}")
    print("---")

In [ ]:
# clean cache ?

import os
from pathlib import Path
import datasets
import huggingface_hub

def find_all_cache_locations():
    """Find all possible Hugging Face cache locations"""
    
    print("=== HUGGING FACE CACHE LOCATIONS ===\n")
    
    # 1. Datasets cache
    datasets_cache = datasets.config.HF_DATASETS_CACHE
    print(f"1. Datasets cache directory:")
    print(f"   {datasets_cache}")
    print(f"   Exists: {Path(datasets_cache).exists()}")
    if Path(datasets_cache).exists():
        items = list(Path(datasets_cache).iterdir())
        print(f"   Items: {len(items)}")
        for item in items[:5]:  # Show first 5 items
            print(f"   - {item.name}")
        if len(items) > 5:
            print(f"   ... and {len(items) - 5} more")
    print()
    
    # 2. Hub cache (for models and datasets)
    hub_cache = huggingface_hub.constants.HF_HUB_CACHE
    print(f"2. Hub cache directory:")
    print(f"   {hub_cache}")
    print(f"   Exists: {Path(hub_cache).exists()}")
    if Path(hub_cache).exists():
        items = list(Path(hub_cache).iterdir())
        print(f"   Items: {len(items)}")
        for item in items[:5]:
            print(f"   - {item.name}")
        if len(items) > 5:
            print(f"   ... and {len(items) - 5} more")
    print()
    
    # 3. Check environment variables
    print("3. Environment variables:")
    env_vars = [
        'HF_DATASETS_CACHE', 'HF_HUB_CACHE', 'HF_HOME', 
        'HUGGINGFACE_HUB_CACHE', 'TRANSFORMERS_CACHE'
    ]
    for var in env_vars:
        value = os.environ.get(var, 'Not set')
        print(f"   {var}: {value}")
    print()
    
    # 4. Common cache locations to check
    print("4. Other common cache locations:")
    common_locations = [
        Path.home() / ".cache" / "huggingface",
        Path.home() / ".cache" / "huggingface" / "datasets", 
        Path.home() / ".cache" / "huggingface" / "hub",
        Path("~/.huggingface").expanduser(),
        Path.cwd() / ".cache",
    ]
    
    for location in common_locations:
        exists = location.exists()
        print(f"   {location}: {'EXISTS' if exists else 'Not found'}")
        if exists:
            try:
                items = list(location.iterdir())
                print(f"     Items: {len(items)}")
                # Look for dataset-specific folders
                for item in items:
                    if 'd4rk3r' in item.name.lower() or 'resumes' in item.name.lower():
                        print(f"     FOUND DATASET CACHE: {item}")
            except PermissionError:
                print(f"     (Permission denied)")
    print()

# Run the function
find_all_cache_locations()

# Also check the current working directory for any hidden cache
print("5. Checking current working directory for cache:")
cwd = Path.cwd()
for item in cwd.iterdir():
    if item.is_dir() and ('cache' in item.name.lower() or item.name.startswith('.')):
        print(f"   Found: {item}")
        if 'huggingface' in item.name.lower() or 'datasets' in item.name.lower():
            print(f"   *** This might be your cache! ***")

In [ ]:
# clean cahce

import shutil
from pathlib import Path
import datasets
import huggingface_hub

# Clear datasets cache
datasets_cache = Path(datasets.config.HF_DATASETS_CACHE)
if datasets_cache.exists():
    print(f"Removing datasets cache: {datasets_cache}")
    shutil.rmtree(datasets_cache)

# Clear hub cache  
hub_cache = Path(huggingface_hub.constants.HF_HUB_CACHE)
if hub_cache.exists():
    print(f"Removing hub cache: {hub_cache}")
    shutil.rmtree(hub_cache)

# Clear common cache locations
common_caches = [
    Path.home() / ".cache" / "huggingface",
    Path("~/.huggingface").expanduser(),
]

for cache_path in common_caches:
    if cache_path.exists():
        print(f"Removing: {cache_path}")
        shutil.rmtree(cache_path)

print("All caches cleared!")